<a href="https://colab.research.google.com/github/ljogeiger/nl2sql-gcp/blob/main/%5BExternal%5D_Text_BisonNLtoBQSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inspiration goes to Nikhil Rana (nikhilrana@) https://colab.sandbox.google.com/drive/1vOevDbTmjBVJs7MFwpFZ7y_r6yDtP9zg?usp=sharing#scrollTo=j9BPkBzhTDzn

Author: lukasgeiger@

IMPORTANT NOTE: Do not use this across your production dataset without proper security measures in place. The agent is especially fickle: if it has access to run DML statements across your db, it might issue a DELETE table command and you could loose your data. You could use a Service Account for this or use a test database. Right now, the notebook impersonates your credentials so it has access to do whatever you have access to.

I would also recommend using Colab Enterprise in your GCP environment: https://cloud.google.com/vertex-ai/docs/colab/create-console-quickstart

In [ ]:
#@title Packages
!pip install langchain
!pip install google-cloud-aiplatform
!pip install langchain langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Create SQL engine for BigQuery
!pip install sqlalchemy==1.4.49
!pip install sqlalchemy-bigquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.20
    Uninstalling SQLAlchemy-2.0.20:
      Successfully uninstalled SQLAlchemy-2.0.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


In [ ]:
#@title Google Cloud Authorization
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
#@title Import Packages
from langchain.llms import VertexAI

In [ ]:
#@title Test LLM Text-Bison
llm = VertexAI(project="<project-name>", location="us-central1", model_name="text-bison")
llm("Explain large language models in two sentence")

' A large language model (LLM) is a type of artificial intelligence (AI) that can understand and generate human language. \n\nLLMs are trained on massive datasets of text and code, and they can be used for a variety of tasks, such as translation, summarization, question answering, and creative writing.'

In [ ]:
# @title BigQuery Client Setup
%load_ext google.cloud.bigquery

from google.cloud import bigquery
from google.colab import data_table

project = '<project-id>' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [ ]:
%%bigquery dei_df --project $project

# NOTE: cannot import all 26 GB of data as there is not enough RAM. Therefore need to limit the query.
# SELECT * FROM `cloud-llm-preview4.lukasgeiger_marketcheck.new_cars`

SELECT * FROM `bigquery-public-data.imdb.reviews` LIMIT 100

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
dei_df

In [ ]:
dei_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   review           100 non-null    object
 1   split            100 non-null    object
 2   label            100 non-null    object
 3   movie_id         100 non-null    object
 4   reviewer_rating  100 non-null    Int64 
 5   movie_url        100 non-null    object
 6   title            0 non-null      object
dtypes: Int64(1), object(6)
memory usage: 5.7+ KB


In [ ]:
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [ ]:
engine = create_engine('bigquery://<project-id>/imdb')

In [ ]:
engine.execute('SELECT * FROM `<project-id>.imdb.reviews`').first()

('I had to see this on the British Airways plane. It was terribly bad acting and a dumb story. Not even a kid would enjoy this. Something to switch off if possible.', 'test', 'Negative', 'tt0158887', 2, 'http://www.imdb.com/title/tt0158887/', None)

In [ ]:
# @title Create a SQLDatabase instance from the engine

from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=['reviews','title_basics'])

In [ ]:
# @title Create SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
# @title Using GoogleSQL Prompt template to guide the LLM to generate output

from langchain.prompts.prompt import PromptTemplate

_googlesql_prompt = """You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run. This will be the SQLQuery. Then look at the results of the query. This will be SQLResults. Finally return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
When running SQLQuery across BigQuery you must only include BigQuery SQL code from SQLQuery.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}"""

GOOGLESQL_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=_googlesql_prompt,
)

In [ ]:
# @title Testing 1: Top 5 ratings and reviews

final_prompt = GOOGLESQL_PROMPT.format(input='give me the top 5 ratings and the corresponding reviews', table_info='imdb.reviews', top_k=100)
db_chain.run(final_prompt)



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run. This will be the SQLQuery. Then look at the results of the query. This will be SQLResults. Finally return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 100 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
When running SQLQuery across BigQuery you must only include BigQuery SQL code from SQLQuery.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which tabl

"10: This movie is amazing. You will NEVER laugh harder. It's a target. No, I think it's...yes it's...A BOOB! This movie gets funnier by the second--like when Jackie Chan's character finally dies in his final fight scene. This movie is velly velly seekwet like treasha! Congrats if you buy or rent...\n10: Insanely well crafted mini-series.I recall seeing most of it twice when shown on American Playhouse on PBS. Was heavily promoted at the time. I believe it might have been one of the very early mini"

In [ ]:
# @title Testing 2: Negative reviews

final_prompt = GOOGLESQL_PROMPT.format(input='How many negative reviews are there?', table_info='imdb.reviews', top_k=10)
db_chain.run(final_prompt)



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run. This will be the SQLQuery. Then look at the results of the query. This will be SQLResults. Finally return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
When running SQLQuery across BigQuery you must only include BigQuery SQL code from SQLQuery.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table

'25000'

In [ ]:
# Verifying the answer
%%bigquery --project $project

SELECT COUNT(*) AS negative_review_count
FROM imdb.reviews
WHERE label = 'Negative'

Query is running:   0%|          |

Downloading:   0%|          |

,negative_review_count
0,25000


In [ ]:
# @title Testing 3: with Langchain SQL agent.

from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatVertexAI

from langchain.llms.openai import OpenAI

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=VertexAI(model_name = "text-bison"))

In [ ]:
agent_executor = create_sql_agent(
    llm=VertexAI(model_name = "text-bison"),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    format_instructions='''Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of SQLDatabaseToolkit
    Action Input: the input to the action
    Observation: the result of the action


    The final resonse will be in the format:
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Remember you must follow the above format when formulating your response. Alway preface your response by either Thought, Action, Action Input, or Observation.

    Here is an example:

    Thought: I need to find out the population of Canada in 2023
    Action: Search
    Action Input: Population of Canada in 2023

    Observation:The current population of Canada is 38,658,314 as of Wednesday, April 12, 2023, based on Worldometer elaboration of the latest United Nations data. I now know the final answer
    Final Answer: Arrr, there be 38,658,314 people livin' in Canada as of 2023!

    ''',
)

In [ ]:
agent_executor('Give me the top 5 movie titles based on reviewer rating from the reviews table and then combine it with title_basics table to get me the runtime of the movies.')



> Entering new AgentExecutor chain...
 Action: sql_db_list_tables
Action Input: 
Observation: reviews, title_basics
Thought: I need to find the top 5 movie titles based on reviewer rating from the reviews table. Then, I need to combine the results with the title_basics table to get the runtime of the movies.
Action: sql_db_query_checker
Action Input: SELECT t1.title, t2.runtime 
FROM reviews t1 
JOIN title_basics t2 ON t1.tconst = t2.tconst 
ORDER BY t1.averageRating DESC 
LIMIT 5;
Observation:  ```sql
SELECT t1.title, t2.runtime 
FROM reviews t1 
JOIN title_basics t2 ON t1.tconst = t2.tconst 
ORDER BY t1.averageRating DESC 
LIMIT 5;
```
Thought: I need to execute the query and get the results.
Action: sql_db_query
Action Input: SELECT t1.title, t2.runtime 
FROM reviews t1 
JOIN title_basics t2 ON t1.tconst = t2.tconst 
ORDER BY t1.averageRating DESC 
LIMIT 5;
Observation: Error: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 400 Name tconst not found inside t1 at [3:28]

Loc

{'input': 'Give me the top 5 movie titles based on reviewer rating from the reviews table and then combine it with title_basics table to get me the runtime of the movies.',
 'output': 'The top 5 movie titles based on reviewer rating are: \n1. Mighty Like a Moose (22 minutes)\n2. Brown of Harvard (85 minutes)\n3. The Sealed Room (11 minutes)\n4. Show of Shows (128 minutes)\n5. Brown of Harvard (85 minutes)'}

In [ ]:
# Let's verify. Not the same results because they all have value of 10 rating. If rating is the same then the output is random.

%%bigquery --project $project

SELECT t1.title, t2.runtime_minutes AS runtime
FROM imdb.reviews t1
JOIN imdb.title_basics t2 ON t1.movie_id = t2.tconst
ORDER BY t1.reviewer_rating DESC
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,title,runtime
0,Vanity Fair,78
1,Magic Bricks,3
2,The Last Command,88
3,"You're a Sap, Mr. Jap",7
4,The Last Command,88


Notice two things:
1. The agent was able to query against two databases.
2. The agent was also able to handle no column found error and adjusted the query to resolve.